In [1]:
# 데이터 수집 라이브러리 호출
import time
import requests
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

# SQL 연결용 라이브러리 호출
import pymysql

In [2]:
url = 'https://www.vivino.com/explore?e=eJzLLbI11rNQy83MszU0AAK13MQKWxMwK7nS1jtILRlIhKsV2BqqpafZliUWZaaWJOao5Rel2KrlJ1XaqpWXRMcCJcGUEQCDaRfR'

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36')

driver = webdriver.Chrome(options=options)
driver.get(url)
driver.maximize_window()

In [3]:
prev_height = driver.execute_script('return document.body.scrollHeight')
print(f"이전 높이 = {prev_height}")

cnt = 0;
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)

    curr_height = driver.execute_script('return document.body.scrollHeight')

    if curr_height == prev_height:
        driver.execute_script('window.scrollTo(0, 0)')
        break
    cnt += 1
    prev_height = curr_height
print('스크롤 완료', cnt)

이전 높이 = 10466
스크롤 완료 8


In [4]:
soup = BeautifulSoup(driver.page_source, 'lxml')

url_elems = soup.find_all('a', {'class' : 'anchor_anchor__m8Qi- wineCard__cardLink--3F_uB'})
link_list = []

for url in url_elems:
    link_list.append(f"https://www.vivino.com{url['href']}")
    
print(f"총 주소 갯수 {len(link_list)}")

총 주소 갯수 193


In [5]:
# 개별 와인의 url을 받아와서 정보 수집
soups = [] # 개별 와인 page의 url정보가 들어갈 리스트

for i in range(len(link_list)):
    driver = webdriver.Chrome(options=options)
    driver.get(link_list[i])
    driver.maximize_window()
    time.sleep(1)
    prev_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(2)
        
        curr_height = driver.execute_script('return document.body.scrollHeight')

        if curr_height == prev_height:
            driver.execute_script('window.scrollTo(0, 0)')
            break
        prev_height = curr_height
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    soups.append(soup)
    driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.121)


위 코드 약 45분 실행

In [21]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')

cursor = db.cursor()

sql = """
    create table wine_list (
        wine_name varchar(100) not null,
        wine_vintage MEDIUMINT,
        wine_rate decimal(4,2),
        rating_num MEDIUMINT,
        country varchar(100) not null,
        region varchar(100) not null,
        farm varchar(100) not null,
        category varchar(100) not null,
        variety  varchar(100) not null, 
        cost MEDIUMINT,
        tasty_note varchar(100),
        tasty_num MEDIUMINT 
    );
    """

cursor.execute(sql)

0

In [22]:
#와인의 정보 수집
for i in range(len(soups)):
    rate = soups[i].find('div', {'class':'vivinoRating_averageValue__uDdPM'}).text
    
    if soups[i].find('a', {'class':'wine'}) != None:
        name = soups[i].find('a', {'class':'wine'}).text.replace('\n', '').replace('\n', '')
    else:
        name = soups[i].find('span', {'class':'vintage'}).text
    
    country = soups[i].find('div', {'class':'breadCrumbs__breadCrumbs--2pkcX'}).text.split('·')[0],
    region = soups[i].find('div', {'class':'breadCrumbs__breadCrumbs--2pkcX'}).text.split('·')[1],
    farm = soups[i].find('div', {'class':'breadCrumbs__breadCrumbs--2pkcX'}).text.split('·')[2],
    category = soups[i].find('div', {'class':'breadCrumbs__breadCrumbs--2pkcX'}).text.split('·')[3],#와인종류
    variety = soups[i].find('div', {'class':'breadCrumbs__breadCrumbs--2pkcX'}).text.split('·')[4] #포도품종
    # cost = soups[i].find('span', {'class':'purchaseAvailabilityPPC__amount--2_4GT'}).text[1:].replace(',','.')
    vintage = soups[i].find('span', {'class' : 'vintage'}).text[-5:-1]
    rating_num = soups[i].find('div',{'class':'vivinoRating_caption__xL84P'}).text.split(' ')[0]
    
    # 가격정보가 다르게 표기된거 있음
    if soups[i].find('span', {'class':'purchaseAvailabilityPPC__amount--2_4GT'}) != None:
        cost = soups[i].find('span', {'class':'purchaseAvailabilityPPC__amount--2_4GT'}).text[1:].replace(',','.')
    else:
        cost = 0
        
    rate = float(rate)
    name = str(name)
    country = str(country).translate(str.maketrans("()'',","     ")).replace(' ','')
    region = str(region).translate(str.maketrans("()'',","     ")).replace(' ','')
    farm = str(farm).translate(str.maketrans("()'',","     ")).replace(' ','')
    category =str(category).translate(str.maketrans("()'',","     ")).replace(' ','')
    variety = str(variety)
    cost = float(cost)
    rating_num = int(rating_num)
    
    # vinatge가 없는 와인을 존재 if N.V거나 cato로 표기되어 있음
    try:
        vintage = int(vintage)
    except ValueError:
        vintage = 0

    a = soups[i].find_all('span', {'class': 'tasteNote__flavorGroup--1Uaen'})[:3]
    b =  soups[i].find_all('div', {'class': 'tasteNote__mentions--1T_d5'})[:3]

    taste_names = []
    taste_nums = []
    
    for taste_name in a:
        taste_names.append(str(taste_name.text))

    for taste_num in b:
        taste_nums.append(int(taste_num.text.split()[0]))
    # print(taste_names, taste_nums)

    
    if len(a) == 0:
        sql = f"""
                INSERT INTO wine_list(wine_name, wine_vintage, wine_rate, rating_num, country, region, farm, category, variety, cost)
                    VALUES { name, vintage, rate, rating_num, country, region, farm, category, variety, cost },
                           { name, vintage, rate, rating_num, country, region, farm, category, variety, cost },
                           { name, vintage, rate, rating_num, country, region, farm, category, variety, cost };
                """
    else:
        sql = f"""
                INSERT INTO wine_list()
                    VALUES { name, vintage, rate, rating_num, country, region, farm, category, variety, cost, taste_names[0], taste_nums[0] },
                           { name, vintage, rate, rating_num, country, region, farm, category, variety, cost, taste_names[1], taste_nums[1] },
                           { name, vintage, rate, rating_num, country, region, farm, category, variety, cost, taste_names[2], taste_nums[2] };
                """

    cursor.execute(sql)
    db.commit()
